In [1]:
import cloudpickle
import torch

In [20]:
# Load an En-Fr Transformer model trained on WMT'14 data :
model = cloudpickle.load(open('./models.pkl','rb'))
# Use the GPU (optional):
if torch.cuda.is_available():
    model.cuda()

In [21]:
model.translate(['Hello world!','This is clipper world','Wait whaat!'], beam=5)


['Bonjour à tous !', "C'est le monde de la clipper", 'Attendons un whaat !']

In [22]:
next(model.parameters()).is_cuda

True

In [1]:
from clipper_admin import ClipperConnection, DockerContainerManager
from clipper_admin.deployers.pytorch import deploy_pytorch_model
clipper_conn = ClipperConnection(DockerContainerManager())

In [2]:
clipper_conn.connect()
clipper_conn.get_all_apps(), clipper_conn.get_all_models()

20-12-01:05:23:05 INFO     [clipper_admin.py:172] [default-cluster] Successfully connected to Clipper cluster at localhost:1337


(['nmt'], ['nmtfairseq:1'])

In [32]:
# Add an application with a name and an input type
clipper_conn.register_application(name="nmt", input_type="strings", default_output="", slo_micros=1000000 )

20-11-26:07:13:47 INFO     [clipper_admin.py:236] [default-cluster] Application nmt was successfully registered


In [33]:
def translate(model, inputs):
    return model.translate(inputs, beam=5)

translate(model, ['Hello World','This is another world'])

['Bonjour au monde', "C'est un autre monde"]

In [34]:
deploy_pytorch_model(
    clipper_conn,
    name="nmtfa",
    version=1,
    input_type="strings",
    func=translate,
    base_image='custom-image',
    pytorch_model=model)

20-11-26:07:13:49 INFO     [deployer_utils.py:41] Saving function to /tmp/tmpi7e38_nmclipper
20-11-26:07:13:49 INFO     [deployer_utils.py:51] Serialized and supplied predict function
20-11-26:07:14:00 INFO     [pytorch.py:204] Torch model saved
20-11-26:07:14:07 INFO     [clipper_admin.py:534] [default-cluster] Building model Docker image with model data from /tmp/tmpi7e38_nmclipper
20-11-26:07:14:41 INFO     [clipper_admin.py:539] [default-cluster] Step 1/2 : FROM custom-image
20-11-26:07:14:41 INFO     [clipper_admin.py:539] [default-cluster]  ---> db30bc064377
20-11-26:07:14:41 INFO     [clipper_admin.py:539] [default-cluster] Step 2/2 : COPY /tmp/tmpi7e38_nmclipper /model/
20-11-26:07:14:41 INFO     [clipper_admin.py:539] [default-cluster]  ---> f3e843e149da
20-11-26:07:14:41 INFO     [clipper_admin.py:539] [default-cluster] Successfully built f3e843e149da
20-11-26:07:14:41 INFO     [clipper_admin.py:539] [default-cluster] Successfully tagged default-cluster-nmtfa:1
20-11-26:07:14

In [35]:
# Link the model and the app
clipper_conn.link_model_to_app(
    app_name="nmt",
    model_name="nmtfa")

20-11-26:07:14:42 INFO     [clipper_admin.py:303] [default-cluster] Model nmtfa is now linked to application nmt


In [20]:
query_address =clipper_conn.get_query_addr()
query_address

'localhost:1337'

In [27]:
# Run a query
import requests, json, numpy as np
headers = {"Content-type": "application/json"}
requests.post("http://"+query_address+"/nmt/predict", headers=headers, data=json.dumps({
    "input": "[Hello world!]"})).json()
# The result is the same as in the local prediction!

{'query_id': 6, 'output': "b '[Bonjour monde !]'", 'default': False}

In [14]:
clipper_conn.get_linked_models(app_name="nmt")

[]

In [ ]:
clipper_conn.cm.get_num_replicas(name="nmtfairseq", version="1")

In [12]:
clipper_conn.get_clipper_logs()

20-12-01:05:26:21 INFO     [docker_logging_utils.py:19] [default-cluster] Created logging directory: /home/ubuntu/17700_project/clipper/notebooks/clipper_logs


['/home/ubuntu/17700_project/clipper/notebooks/clipper_logs/image_sha256:8a21efc4f5:container_42f9fe1567.log',
 '/home/ubuntu/17700_project/clipper/notebooks/clipper_logs/image_sha256:a25dae3910:container_eb4719c771.log',
 '/home/ubuntu/17700_project/clipper/notebooks/clipper_logs/image_sha256:f859e8beec:container_6dd016d0cd.log',
 '/home/ubuntu/17700_project/clipper/notebooks/clipper_logs/image_sha256:f1f579ef14:container_2f987cc2df.log',
 '/home/ubuntu/17700_project/clipper/notebooks/clipper_logs/image_sha256:116b17ebda:container_541a3d965c.log',
 '/home/ubuntu/17700_project/clipper/notebooks/clipper_logs/image_sha256:c1949ec48c:container_823c5a58f7.log']

In [51]:
!docker ps -a --filter label=ai.clipper.container.label

CONTAINER ID        IMAGE                               COMMAND                  CREATED             STATUS                   PORTS                                            NAMES
20b80636d976        default-cluster-nmtfa:1             "/container/containe…"   5 minutes ago       Up 5 minutes (healthy)                                                    nmtfa_1-65358
c04c3614cc4a        prom/prometheus:v2.9.2              "/bin/prometheus --c…"   2 hours ago         Up 2 hours               0.0.0.0:9090->9090/tcp                           metric_frontend-30229
07b02e784694        clipper/frontend-exporter:0.4.1     "python /usr/src/app…"   2 hours ago         Up 2 hours                                                                query_frontend_exporter-34886
d1bcf9160443        clipper/query_frontend:0.4.1        "/clipper/query_fron…"   2 hours ago         Up 2 hours               0.0.0.0:1337->1337/tcp, 0.0.0.0:7000->7000/tcp   query_frontend-34886
5a59edaa3488        clipper/mana

In [31]:
print(clipper_conn.unlink_model_from_app(model_name="nmtfa", app_name="nmt"))
print(clipper_conn.get_all_models())
print(clipper_conn.stop_all_model_containers())
print(clipper_conn.delete_application('nmt'))

20-11-26:07:13:20 INFO     [clipper_admin.py:344] Model nmtfa is now removed to application nmt


None
['nmtfa:1']


20-11-26:07:13:31 INFO     [clipper_admin.py:1285] Model nmtfa:1 was successfully deleted
20-11-26:07:13:31 INFO     [clipper_admin.py:1410] [default-cluster] Stopped all Clipper model containers:
{'nmtfa': ['1']}
20-11-26:07:13:31 WARNING  [clipper_admin.py:240] [default-cluster] [DEPRECATED] Use 'unregister_application' API instead of this.
20-11-26:07:13:31 INFO     [clipper_admin.py:260] [default-cluster] Application nmt was successfully unregistered


None
None


In [38]:
!docker rm $(docker ps -a -q)

ea855d86a9ea
cf8b9e259aac
Error response from daemon: You cannot remove a running container c04c3614cc4ae5e6019ee39965709f35aab60ea518a97b22aa53902cdb036b80. Stop the container before attempting removal or force remove
Error response from daemon: You cannot remove a running container 07b02e7846946ba748611d424a4516af68c177d6331fa9ad470cf3c1dd7a9ec5. Stop the container before attempting removal or force remove
Error response from daemon: You cannot remove a running container d1bcf9160443e08eabc7f86e083b6d7083494a3308b13d867037b0b730ea5e5a. Stop the container before attempting removal or force remove
Error response from daemon: You cannot remove a running container 5a59edaa3488a8561362fcc662dc0b70585da18a5819cd3ee164c11140e87337. Stop the container before attempting removal or force remove
Error response from daemon: You cannot remove a running container 95814de241e17dfa8c426f05df341e78eab62ebe250317ea7e734d76eee001e5. Stop the container before attempting removal or force remove


In [49]:
clipper_conn.inspect_instance()

{'counters': [{'app:nmt:num_predictions': {'count': '14'}},
  {'model:nmtfa:1:num_predictions': {'count': '0'}},
  {'app:nmt:num_predictions': {'count': '0'}},
  {'app:nmt:num_predictions': {'count': '0'}},
  {'internal:prediction_cache_lookups': {'count': '11'}},
  {'internal:aggregate_num_predictions': {'count': '14'}}],
 'ratio_counters': [{'internal:prediction_cache_hit_ratio': {'ratio': '0.90909090909090906'}},
  {'app:nmt:default_prediction_ratio': {'ratio': '0.21428571428571427'}},
  {'app:nmt:default_prediction_ratio': {'ratio': 'nan'}},
  {'app:nmt:default_prediction_ratio': {'ratio': 'nan'}},
  {'model:nmtfa:1:cache_hit_ratio': {'ratio': '1'}}],
 'meters': [{'app:nmt:prediction_throughput': {'unit': 'events per second',
    'rate': '0.019831888677749945',
    'rate_1min': '0.004151615425657916',
    'rate_5min': '0.017115244038300805',
    'rate_15min': '0.010776043453515138'}},
  {'app:nmt:prediction_throughput': {'unit': 'events per second',
    'rate': '0',
    'rate_1min'